In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from typing import Any, Optional

from langsmith import Client
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langsmith.evaluation import LangChainStringEvaluator
from langchain.smith import EvaluationConfig, run_on_dataset

# --- Setup ---
load_dotenv()
client = Client()

# --- Prerequisite: Run the Experiment from Lesson 3 ---
# This part creates the experiment we need to analyze.
print("--- Running Prerequisite Experiment (from Lesson 3) ---")
experiment_project_name = "Experiment: Final Local Model Comparison"

def is_concise(run, example) -> dict:
    llm_output = run.outputs.get("text")
    if llm_output is None: return {"key": "is_concise", "score": 0}
    return {"key": "is_concise", "score": int(len(llm_output) < 75)}

custom_evaluator = LangChainStringEvaluator([is_concise])
eval_config = EvaluationConfig(evaluators=[custom_evaluator])
summary_prompt = PromptTemplate.from_template("Summarize the following article in one sentence: {article}")

# Run for GPT-3.5-Turbo
summary_chain_gpt3 = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0), prompt=summary_prompt)
run_on_dataset(client=client, dataset_name="Article Summaries", llm_or_chain_factory=summary_chain_gpt3, evaluation=eval_config, project_name=experiment_project_name)
print("Finished evaluation for gpt-3.5-turbo.")

# Run for GPT-4o
summary_chain_gpt4 = LLMChain(llm=ChatOpenAI(model="gpt-4o", temperature=0), prompt=summary_prompt)
run_on_dataset(client=client, dataset_name="Article Summaries", llm_or_chain_factory=summary_chain_gpt4, evaluation=eval_config, project_name=experiment_project_name)
print("Finished evaluation for gpt-4o.")
print(f"\nExperiment complete! View results in project: '{experiment_project_name}'")
print("-" * 50)


# --- Lesson 4: Analyze Experiment Results ---
# This is the new part for this lesson.
print("\n--- Analyzing Experiment Results (Lesson 4) ---")

# Use a try-except block to handle potential errors gracefully
try:
    # Retrieve the results from the experiment we just ran
    experiment_results = client.get_experiment_results(project_name=experiment_project_name)

    # Display the results in a structured table using pandas
    results_df = pd.DataFrame(experiment_results["results"])
    print("Displaying experiment results:")
    display(results_df)
    print("\n✅ Analysis complete.")

except Exception as e:
    print(f"\n❌ FAILED to analyze results due to an error: {e}")

ImportError: cannot import name 'EvaluationConfig' from 'langchain.smith' (c:\Users\dell\Documents\CODES\LangSmith\ls-academy\Lib\site-packages\langchain\smith\__init__.py)